<img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/2e/Logo_der_Technischen_Universit%C3%A4t_Berlin.svg/200px-Logo_der_Technischen_Universit%C3%A4t_Berlin.svg.png" decoding="async" width="150" height="112">

# Python for ecohydrology

Prof. Dr. Eva Paton & Dr. Pedro Alencar


## Python - Crop Analysis

This notebook presents:

* WOFOST/PCSE

WOrld FOod STudies - WOFOST is a simulation model of growth and produciton of annual field crops. It is one of the key parts of the Global Yield Gap Atlas ([GYGA](http://www.yieldgap.org/)) and Monitoring Agricultural ResourceS ([MARS](https://joint-research-centre.ec.europa.eu/monitoring-agricultural-resources-mars_en)) initiatives.

It can be easily accessed with the Python Crop Simulation Environment ([PCSE](https://pcse.readthedocs.io/en/stable/#)).

For more information about WOFOST, visit their main [page](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost.htm), where tips on [implementation](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/implementation-of-wofost.htm), and [downloads](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/downloads-wofost.htm), and [documentation](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/documentation-wofost.htm) are available.

To use WOFOST in Python, first it is necessary to install it:

In [ ]:
pip install pcse

**Let's first import a few packages**

In [ ]:
import sys, os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime

from IPython import display

In [ ]:
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

It is important to define a path to all input files. Make sure that the [folder](https://github.com/pedroalencar1/IntroToPython) `Data_WOFOST` is placed in a know location in your computer.

In [ ]:
data_dir = os.path.join("path/to/data/folder")
# data_dir = os.path.join("")

In [ ]:
data_dir = os.path.join("/Users/alencar/Library/CloudStorage/OneDrive-Personal/@_PostDoc/IntroToPython/Data_WOFOST")

In the data folder there are miltiple sub-folders with input data of WOFOST.

1. agro: has the agromanagement files that provide to the model the crop calendar and management activities (if any)
2. crop: has the crop parameter files for multiple calibrated crops, such as potato, wheat and corn
3. meteo: has the weather input files, containing geographic location and meteorological variables P, Tmax, Tmin, Irrad, WS.
4. soil: has the standard files with soil parameters


_Obs: sub-folders `site` and `output` will not be used now._

On soil parameters:
<img alt="" src="https://raw.githubusercontent.com/pedroalencar1/IntroToPython/master/Soil_parameters.png" decoding="async" width="850">

### 1. Read paramenters

#### **2.1. Crop parameters**

Here we can list all available crops in the WOFOST files

In [ ]:
from pcse.fileinput import YAMLCropDataProvider
cropd = YAMLCropDataProvider()

cropd.print_crops_varieties()

In [ ]:
cropd = YAMLCropDataProvider(fpath = os.path.join(data_dir,"crop"))
# cropd.print_crops_varieties()

cropd.set_active_crop('maize', 'Grain_maize_201')
# print(yaml.dump(cropd, sort_keys=False, default_flow_style=False))

#### **2.2. Soil parameters**

The default values of soil parameters of WOFOST are derived from the European Soil Database V2.0 that separate the soils in 5 categoties, based on the containts of clay (C) and sand (S): 

1. Coarse (C < 18% and S > 65%)
2. Medium (18 < C < 35% and S > 15% or C < 18% and 15 < S < 65%)
3. Medium fine (C < 35% and S < 15%)
4. Fine (35 < C < 60%)
5. Very fine (C > 60%)

Additionally there is a class for Peat soils (no mineral texture, designated by the number *9* and not available in WOFOST)

For more, visit the [European soil satabase maps](https://esdac.jrc.ec.europa.eu/resource-type/european-soil-database-maps#)

**Note:** Soil class is selected by directly providing an input file (available at the [WOFOST GitHub page](https://github.com/ajwdewit/WOFOST/tree/master/soild)). The default values are loaded as a dictionary. The values then can be changed individually to locally calibrated parameters (if available) or for synthetic values (to assess uncertainty).

In [ ]:
from pcse.fileinput import CABOFileReader

soilfile = os.path.join(data_dir, 'soil', 'EC1.NEW')
soild = CABOFileReader(soilfile)

#### 2.3 Site parameters

Here we provide a few parameters relevant to the site location. Typically:
* WAV - Initial amount of water in rootzone *in excess of wilting point* in cm. Value is limited to SMLIM
* SMLIM - Maximum initial soil moisture in rooted zone (will be forced between SMW and SM0) (cm$^3$.cm$^{-3}$)
* CO2 - Atmospheric CO2 level in ppm. Default 360

In [ ]:
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=10) # not updating the dictionary with the new key
sited['CO2'] = 360
print(sited)

#### 2.4. Packaging all parameters

_"Finally, we need to pack the different sets of parameters into one variable using the `ParameterProvider`. This is needed because PCSE expects one variable that contains all parameter values."_

In [ ]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)


**Note:** The parameters values are loaded as a dictionary. The values then can be easily changed here for fine tuning  and tests

To assess each one of the parameters dictionaries use:

* Soil: parameters._soildata
* Crop: parameters._cropdata
* Site: parameters._sitedata

In [ ]:
parameters._sitedata

Also at this stage you can easily change values of parameters

In [ ]:
parameters._sitedata['CO2'] = 396

parameters._sitedata

In [ ]:
parameters._soildata['RDMSOL'] = 120 #max root depth

### 3. Read agromanagement

The Agromanagement file provides to WOFOST basic information on the begining and end of the simulation for each year. 

In [ ]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'basic.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

The agromanagement file can be edited as a dictionary

In [ ]:
year = 2019

In [ ]:
agromanagement[0][datetime.date(year, 1, 1)] = agromanagement[0][datetime.date(9999, 1, 1)]
agromanagement[0].pop(datetime.date(9999, 1, 1))

print(agromanagement)

In [ ]:
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_name'] = parameters._cropdata.current_crop_name
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['variety_name'] = parameters._cropdata.current_variety_name
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_start_date'] = datetime.date(year, 4, 24)
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_start_type'] = 'sowing' #or "emergence"
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_end_date'] = datetime.date(year, 11, 26)
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_end_type'] = "harvest" #or "maturity"
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['max_duration'] = 264

print(agromanagement)

### 3. Weather data

Currently the easiest way to load weather data into WOFOST is using Excel files. A script in `R` was prepared to help generating these files from DWD data, however, it is not possible to read them directly, as most cells are interpreted as text instead of numeric.
To solve this, the solution is to manually copy the generated files in R into a original (`basic_weather.xls`) with attention to make all cells as numeric (except dates).

In [ ]:
from pcse.fileinput import ExcelWeatherDataProvider

weatherfile = os.path.join(data_dir, 'meteo', 'potsdam1.xls')
wdp = ExcelWeatherDataProvider(weatherfile)

print(wdp)

### 4. Processing WOFOST

Once the parameters and set, weather data provider loaded and agromanagement defined, the model can be run. 

Here we process the water limited and potential production scenarios

In [ ]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim_wlp = Wofost72_WLP_FD(parameters, wdp, agromanagement) # water limited production
wofsim_pp = Wofost72_PP(parameters, wdp, agromanagement) # potential production

In [ ]:
wofsim_wlp.run_till_terminate()
df_results_wlp = pd.DataFrame(wofsim_wlp.get_output())
df_results_wlp = df_results_wlp.set_index("day")
df_results_wlp.tail()

In [ ]:
wofsim_pp.run_till_terminate()
df_results_pp = pd.DataFrame(wofsim_pp.get_output())
df_results_pp = df_results_pp.set_index("day")
df_results_pp.tail()

**Visualization**

In [ ]:
df_results_wlp['type'] = 'wlp'
df_results_pp['type'] = 'pp'

df = pd.concat([df_results_wlp, df_results_pp]).reset_index(drop=False)

df.tail()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.25)

ax = fig.add_subplot(2, 2, 1)
sns.lineplot(data=df, x = 'day', y = 'TAGP', hue = 'type', 
            palette = ['red', 'blue']).set(title='TAPG Gap {}'.format(year), xlabel = None, xticklabels=[])

ax = fig.add_subplot(2, 2, 2)
sns.lineplot(data=df, x = 'day', y = 'LAI', hue = 'type', 
            palette = ['red', 'blue']).set(title='LAI Gap {}'.format(year), xlabel = None, xticklabels=[])

ax = fig.add_subplot(2, 2, 3)
sns.lineplot(data=df, x = 'day', y = 'TRA', hue = 'type', 
            palette = ['red', 'blue']).set(title='TRA Gap {}'.format(year),xlabel = 'Month', xticklabels=[5,6,7,8,9,10,11,12])

ax = fig.add_subplot(2, 2, 4)
sns.lineplot(data=df, x = 'day', y = 'TWRT', hue = 'type', 
            palette = ['red', 'blue']).set(title='TWRT Gap {}'.format(year),xlabel = 'Month', xticklabels=[5,6,7,8,9,10,11,12])
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})
sns.lineplot(data=df, x = 'day', y = 'SM', hue = 'type', 
            palette = ['red', 'blue']).set(title='SW Gap {}'.format(year),xlabel = 'Month')